In [1]:
from __future__ import print_function
from openpathsampling.experimental.storage.ops_storage import OPSStorage, ops_class_info, ops_schema
from openpathsampling.experimental.storage.sql_backend import SQLStorageBackend
import numpy as np
from openpathsampling.experimental.storage.serialization_helpers import get_uuid

import collections

In [2]:
import logging
import sys

root = logging.getLogger()
root.setLevel(logging.DEBUG)

ch = logging.StreamHandler(sys.stdout)
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
root.addHandler(ch)

In [3]:
backend = SQLStorageBackend("test.sql", mode='w')
storage = OPSStorage.from_backend(
    backend=backend,
    schema=ops_schema,
    class_info=ops_class_info
)

2020-07-14 18:07:38,435 - openpathsampling.experimental.storage.sql_backend - INFO - Add schema table uuid
2020-07-14 18:07:38,437 - openpathsampling.experimental.storage.sql_backend - INFO - Add schema table tables
2020-07-14 18:07:38,445 - openpathsampling.experimental.storage.sql_backend - INFO - Add schema table samples
2020-07-14 18:07:38,460 - openpathsampling.experimental.storage.sql_backend - INFO - Add schema table sample_sets
2020-07-14 18:07:38,468 - openpathsampling.experimental.storage.sql_backend - INFO - Add schema table trajectories
2020-07-14 18:07:38,479 - openpathsampling.experimental.storage.sql_backend - INFO - Add schema table move_changes
2020-07-14 18:07:38,531 - openpathsampling.experimental.storage.sql_backend - INFO - Add schema table steps
2020-07-14 18:07:38,552 - openpathsampling.experimental.storage.sql_backend - INFO - Add schema table details
2020-07-14 18:07:38,583 - openpathsampling.experimental.storage.sql_backend - INFO - Add schema table storable_f

In [4]:
import openpathsampling as paths
logging.getLogger('openpathsampling').setLevel(logging.CRITICAL)
old_storage = paths.Storage("../toy_mstis_1k_OPS1_py36.nc", 'r')

In [5]:
step = old_storage.steps[-1]

In [6]:
step_uuid = get_uuid(step)
step_uuid

'205825572803023860022845415596769487056'

In [7]:
from openpathsampling.experimental.storage.serialization_helpers import get_all_uuids

In [8]:
#sum([len(s) for s in sample_set])

In [9]:
step.to_dict()

{'mccycle': 1000,
 'previous': <openpathsampling.sample.SampleSet at 0x1231597f0>,
 'change': <openpathsampling.movechange.PathSimulatorMoveChange at 0x125a87400>,
 'active': <openpathsampling.sample.SampleSet at 0x123159320>,
 'simulation': <openpathsampling.pathsimulators.path_sampling.PathSampling at 0x125a87978>}

These take forever because they're still using the old OPS loading process. They have to load all the lazy objects. It takes 30 seconds for the `step.active`! This is ridiculous.

In [10]:
%%time
get_all_uuids(step.active);

CPU times: user 15 s, sys: 1.24 s, total: 16.2 s
Wall time: 16.3 s


{'205825572803023860022845415596769487054': <openpathsampling.sample.SampleSet at 0x123159320>,
 '205825572803023860022845415596769375132': <Sample @ 0x121ac4ac8>,
 '205825572803023860022845415596769426136': <Sample @ 0x1231599e8>,
 '205825572803023860022845415596769442906': <Sample @ 0x123159a20>,
 '205825572803023860022845415596769447010': <Sample @ 0x125a87080>,
 '205825572803023860022845415596769453502': <Sample @ 0x125a871d0>,
 '205825572803023860022845415596769467356': <Sample @ 0x125a87240>,
 '205825572803023860022845415596769470120': <Sample @ 0x125a87208>,
 '205825572803023860022845415596769472296': <Sample @ 0x125a872b0>,
 '205825572803023860022845415596769473070': <Sample @ 0x125a87278>,
 '205825572803023860022845415596769480946': <Sample @ 0x125a872e8>,
 '205825572803023860022845415596769485076': <Sample @ 0x121f8df60>,
 '205825572803023860022845415596769486208': <Sample @ 0x125a87198>,
 '205825572803023860022845415596769486788': <Sample @ 0x125a87470>,
 '205825572803023860

In [11]:
get_all_uuids(step.previous);

In [12]:
all_uuids = get_all_uuids(step.change)

In [13]:
print(len(all_uuids))

1617


In [14]:
step.change.to_dict()

{'mover': <openpathsampling.pathmover.PathSimulatorMover at 0x125a875c0>,
 'details': <openpathsampling.pathmover.Details at 0x126417948>,
 'samples': [],
 'input_samples': [],
 'subchanges': [<openpathsampling.movechange.RandomChoiceMoveChange at 0x125a87518>],
 'cls': 'PathSimulatorMoveChange'}

In [15]:
step.change.mover.to_dict()

{'mover': <openpathsampling.pathmover.RandomChoiceMover at 0x125fc4ba8>,
 'pathsimulator': <openpathsampling.pathsimulators.path_sampling.PathSampling at 0x125a87978>}

In [16]:
step.simulation.root_mover.movers[0].movers[0].to_dict()

{'ensemble1': <openpathsampling.ensemble.TISEnsemble at 0x111df36a0>,
 'ensemble2': <openpathsampling.ensemble.TISEnsemble at 0x125236198>,
 'bias': None}

In [17]:
get_all_uuids(step.simulation.root_mover.movers[0].movers[0])

{'205825572803023860022845415596767639426': <openpathsampling.pathmover.ReplicaExchangeMover at 0x125a87b00>,
 '205825572803023860022845415596767576816': <openpathsampling.ensemble.TISEnsemble at 0x111df36a0>,
 '205825572803023860022845415596767576846': <openpathsampling.ensemble.TISEnsemble at 0x125236198>,
 '205825572803023860022845415596767576740': <openpathsampling.volume.CVDefinedVolume at 0x1252364e0>,
 '174395458874316017953765140704788480026': <openpathsampling.collectivevariable.CoordinateFunctionCV at 0x125236358>,
 '46380510405785918200155421121190035598': <openpathsampling.ensemble.IntersectionEnsemble at 0x1259c16d8>,
 '205825572803023860022845415596767576742': <openpathsampling.volume.CVDefinedVolume at 0x1259c19b0>,
 '46380510405785918200155421121190035664': <openpathsampling.ensemble.IntersectionEnsemble at 0x1259ca898>,
 '205825572803023860022845415596767576786': <openpathsampling.volume.UnionVolume at 0x111df3a20>,
 '174395458874316017953765140704788480032': <openpath

In [18]:
#%load_ext line_profiler

In [19]:
from openpathsampling.experimental.storage.serialization_helpers import default_find_uuids

In [20]:
#%%time
#%lprun -f get_all_uuids 
all_u = get_all_uuids(step)
#excluded_iterables=(np.ndarray, paths.Sample, paths.SampleSet, paths.MoveChange))

In [21]:
len(all_u)

2981

In [22]:
collections.Counter([v.__class__ for v in all_u.values()])

Counter({openpathsampling.pathsimulators.path_simulator.MCStep: 1,
         openpathsampling.sample.SampleSet: 3,
         openpathsampling.movechange.PathSimulatorMoveChange: 1,
         openpathsampling.pathsimulators.path_sampling.PathSampling: 1,
         openpathsampling.sample.Sample: 27,
         openpathsampling.pathmover.PathSimulatorMover: 1,
         openpathsampling.pathmover.Details: 4,
         openpathsampling.movechange.RandomChoiceMoveChange: 2,
         openpathsampling.movechange.AcceptedSampleMoveChange: 1,
         openpathsampling.high_level.move_scheme.DefaultScheme: 1,
         openpathsampling.pathmover.RandomChoiceMover: 15,
         openpathsampling.ensemble.MinusInterfaceEnsemble: 3,
         openpathsampling.ensemble.TISEnsemble: 18,
         openpathsampling.ensemble.UnionEnsemble: 2,
         openpathsampling.engines.trajectory.Trajectory: 27,
         openpathsampling.engines.toy.snapshot.ToySnapshot: 2331,
         openpathsampling.pathmover.ReplicaExch

In [23]:
#%%prun -D save.pstat
#%lprun -f SQLStorageBackend.add_to_table 
storage.save(step)

In [24]:
snap = step.active[0].trajectory[0]

In [25]:
backend.metadata.tables['tables']

Table('tables', MetaData(bind=Engine(sqlite:///test.sql)), Column('name', String(), table=<tables>), Column('idx', Integer(), table=<tables>), Column('module', String(), table=<tables>), Column('class_name', String(), table=<tables>), schema=None)

In [26]:
#storage.load([step_uuid])
storage.load([get_uuid(step)])

In [27]:
sample_set = step.active

In [28]:
sample_set.to_dict()
# note: this is a problem with to_dict-based serialization for data objects

{'samples': [<Sample @ 0x121ac4ac8>,
  <Sample @ 0x1231599e8>,
  <Sample @ 0x123159a20>,
  <Sample @ 0x125a87080>,
  <Sample @ 0x125a871d0>,
  <Sample @ 0x125a87240>,
  <Sample @ 0x125a87208>,
  <Sample @ 0x125a872b0>,
  <Sample @ 0x125a87278>,
  <Sample @ 0x125a872e8>,
  <Sample @ 0x121f8df60>,
  <Sample @ 0x125a87198>,
  <Sample @ 0x125a87470>],
 'ensemble_dict': {<openpathsampling.ensemble.MinusInterfaceEnsemble at 0x125a476a0>: [<Sample @ 0x121ac4ac8>],
  <openpathsampling.ensemble.MinusInterfaceEnsemble at 0x125a32be0>: [<Sample @ 0x1231599e8>],
  <openpathsampling.ensemble.TISEnsemble at 0x1259e6eb8>: [<Sample @ 0x123159a20>],
  <openpathsampling.ensemble.TISEnsemble at 0x1259c1a20>: [<Sample @ 0x125a87080>],
  <openpathsampling.ensemble.TISEnsemble at 0x1259e6f98>: [<Sample @ 0x125a871d0>],
  <openpathsampling.ensemble.MinusInterfaceEnsemble at 0x123159f98>: [<Sample @ 0x125a87240>],
  <openpathsampling.ensemble.TISEnsemble at 0x1259f4048>: [<Sample @ 0x125a87208>],
  <openpaths

In [29]:
sset_info = storage.class_info['sample_sets']
sset_info

ClassInfo(table=sample_sets, cls=<class 'openpathsampling.sample.SampleSet'>, lookup_result=<class 'openpathsampling.sample.SampleSet'>, find_uuids=<openpathsampling.experimental.storage.serialization_helpers.SchemaFindUUIDs object at 0x121a47c88>)

In [30]:
sset_info.serializer(sample_set)

{'samples': '["UUID(205825572803023860022845415596769375132)", "UUID(205825572803023860022845415596769426136)", "UUID(205825572803023860022845415596769442906)", "UUID(205825572803023860022845415596769447010)", "UUID(205825572803023860022845415596769453502)", "UUID(205825572803023860022845415596769467356)", "UUID(205825572803023860022845415596769470120)", "UUID(205825572803023860022845415596769472296)", "UUID(205825572803023860022845415596769473070)", "UUID(205825572803023860022845415596769480946)", "UUID(205825572803023860022845415596769485076)", "UUID(205825572803023860022845415596769486208)", "UUID(205825572803023860022845415596769486788)"]',
 'uuid': '205825572803023860022845415596769487054'}

In [31]:
samp = sample_set[0]

In [32]:
samp.__uuid__

205825572803023860022845415596769472296

In [33]:
collections.Counter([v.__class__ for v in get_all_uuids(samp).values()])
# TESTS: make sure that a simple sample gets everything!

Counter({openpathsampling.sample.Sample: 1,
         openpathsampling.ensemble.TISEnsemble: 1,
         openpathsampling.engines.trajectory.Trajectory: 1,
         openpathsampling.engines.toy.snapshot.ToySnapshot: 4,
         openpathsampling.volume.CVDefinedVolume: 4,
         openpathsampling.collectivevariable.CoordinateFunctionCV: 3,
         openpathsampling.ensemble.IntersectionEnsemble: 6,
         openpathsampling.engines.toy.engine.ToyEngine: 1,
         openpathsampling.volume.UnionVolume: 5,
         openpathsampling.ensemble.SequentialEnsemble: 1,
         openpathsampling.ensemble.PartOutXEnsemble: 2,
         openpathsampling.engines.toy.topology.ToyTopology: 1,
         openpathsampling.ensemble.AllInXEnsemble: 4,
         openpathsampling.ensemble.LengthEnsemble: 4,
         openpathsampling.ensemble.AllOutXEnsemble: 2,
         openpathsampling.engines.toy.integrators.LangevinBAOABIntegrator: 1,
         openpathsampling.engines.toy.pes.PES_Add: 3,
         openpathsa

In [34]:
traj = samp.trajectory

In [35]:
collections.Counter([v.__class__ for v in get_all_uuids(traj).values()])

Counter({openpathsampling.engines.trajectory.Trajectory: 1,
         openpathsampling.engines.toy.snapshot.ToySnapshot: 4,
         openpathsampling.engines.toy.engine.ToyEngine: 1,
         openpathsampling.engines.toy.topology.ToyTopology: 1,
         openpathsampling.engines.toy.integrators.LangevinBAOABIntegrator: 1,
         openpathsampling.engines.toy.pes.PES_Add: 3,
         openpathsampling.engines.toy.pes.Gaussian: 3,
         openpathsampling.engines.toy.pes.OuterWalls: 1})

In [36]:
sset_uuid = get_uuid(sample_set)
sset_uuid

'205825572803023860022845415596769487054'

In [37]:
#%%prun -D load.pstat
storage.load([sset_uuid])

In [38]:
#backend.load_uuids_table(['118336820345834620840518008770335453780'])

In [39]:
sample_set[0].ensemble.__uuid__

205825572803023860022845415596767576816

In [40]:
sample_set.__uuid__

205825572803023860022845415596769487054

In [41]:
samp = sample_set.samples[7]

In [42]:
print(samp.ensemble.__uuid__)
print(samp.trajectory.__uuid__)
print(samp.parent.__uuid__)
print(samp.mover.__uuid__)

205825572803023860022845415596767576816
205825572803023860022845415596769472254
205825572803023860022845415596769460492
205825572803023860022845415596767639510


In [43]:
sample_set[0].trajectory[0].xyz

array([[-0.39463076, -0.6676565 ]], dtype=float32)

### Storing objects with no dependents

Objects with no dependents, or which are not explicitly listed as dependents, were not being stored (because they didn't show up in the DAG).

In [44]:
my_full = paths.FullVolume()
full_uuid = get_uuid(my_full)
full_uuid

'46380510405785918200155421121190046606'

In [45]:
storage.save(my_full)

In [46]:
storage.load([full_uuid])

In [47]:
integ = step.active[0].trajectory[0].engine.integ
integ_uuid = get_uuid(integ)

In [48]:
storage.save(integ)

In [49]:
storage.load([integ_uuid])

### Storing and reloading the network

This was having trouble because some dictionaries included mappings for UUID-object keys, which weren't being converted correctly

In [50]:
network = step.simulation.move_scheme.network
network_uuid = get_uuid(network)
network_uuid

'205825572803023860022845415596767576776'

In [51]:
storage.save(network)

In [52]:
loaded_network = storage.load([network_uuid])[0]

In [53]:
print(loaded_network)

Multiple State TIS Network:
TISTransition: Out A
A -> A or all states except A
Interface: 0.0<opA<0.2
Interface: 0.0<opA<0.30000000000000004
Interface: 0.0<opA<0.4
TISTransition: Out B
B -> B or all states except B
Interface: 0.0<opB<0.2
Interface: 0.0<opB<0.30000000000000004
Interface: 0.0<opB<0.4
TISTransition: Out C
C -> C or all states except C
Interface: 0.0<opC<0.2
Interface: 0.0<opC<0.30000000000000004
Interface: 0.0<opC<0.4



### Storing and reloading a PathSimulator

In [54]:
sim = step.simulation
sim_uuid = get_uuid(sim)
sim_uuid

'205825572803023860022845415596767639424'

In [55]:
storage.save(sim)

In [56]:
reloaded_sim = storage.load([sim_uuid])[0]

### Storing and reloading a MoveChange

In [57]:
change = step.change.subchanges[0].subchanges[0].subchanges[0]
change_uuid = get_uuid(change)
change_uuid

'205825572803023860022845415596769487046'

In [58]:
print(change.details.__dict__)

{'__uuid__': 205825572803023860022845415596769487044, 'metropolis_acceptance': 1.0, 'metropolis_random': 0.7953469615000001}


In [59]:
storage.save(change)

In [60]:
list(storage.backend.table_iterator('simulation_objects'))

[(1, '205825572803023860022845415596767639424', '{"root": "UUID(205825572803023860022845415596767639678)", "move_scheme": "UUID(205825572803023860022845415596767639402)", "root_mover": "UUID(205825572803023860022845415596767639648)", "__class__": "PathSampling", "__module__": "openpathsampling.pathsimulators.path_sampling"}', None),
 (2, '205825572803023860022845415596767639650', '{"mover": "UUID(205825572803023860022845415596767639648)", "pathsimulator": "UUID(205825572803023860022845415596767639424)", "__class__": "PathSimulatorMover", "__module__": "openpathsampling.pathmover"}', None),
 (3, '205825572803023860022845415596767639402', '{"movers": {"repex": ["UUID(205825572803023860022845415596767639426)", "UUID(205825572803023860022845415596767639428)", "UUID(20582557280302386002284 ... (6142 characters truncated) ... "root_mover": "UUID(205825572803023860022845415596767639648)", "__class__": "DefaultScheme", "__module__": "openpathsampling.high_level.move_scheme"}', None),
 (4, '205

In [61]:
change.to_dict()

{'mover': <openpathsampling.pathmover.PathReversalMover at 0x125a87ac8>,
 'details': <openpathsampling.pathmover.Details at 0x1264179d8>,
 'samples': [<Sample @ 0x125a87470>],
 'input_samples': [<Sample @ 0x125a87390>],
 'subchanges': [],
 'cls': 'AcceptedSampleMoveChange'}

In [62]:
reloaded_change = storage.load([change_uuid])[0]

In [63]:
print(reloaded_change)

SampleMove : PathReversalMover : True : 1 samples [<Sample @ 0x126af89b0>]


In [64]:
print(reloaded_change.details)

metropolis_acceptance = 1.0
metropolis_random = 0.7953469615000001



### Starting a `PathSampling` simulator

The issue here is that sometimes an object that is expected to be a UUID is actually `None`. Need to be careful with both saving and loading here.

In [65]:
scheme = old_storage.schemes[0]
sample_set = step.active

In [66]:
sim = paths.PathSampling(
    move_scheme=scheme,
    sample_set=sample_set,
    storage=storage
)

In [67]:
storage.save(sim)

In [68]:
sim_uuid = get_uuid(sim)
reloaded_sim = storage.load([sim_uuid])[0]

In [69]:
reloaded_sim.to_dict()

{'root': <openpathsampling.sample.SampleSet at 0x126bf8be0>,
 'move_scheme': <openpathsampling.high_level.move_scheme.DefaultScheme at 0x125fc4a58>,
 'root_mover': <openpathsampling.pathmover.RandomChoiceMover at 0x125fc4ba8>}

In [70]:
sim.to_dict()

{'root': <openpathsampling.sample.SampleSet at 0x126bf8be0>,
 'move_scheme': <openpathsampling.high_level.move_scheme.DefaultScheme at 0x125fc4a58>,
 'root_mover': <openpathsampling.pathmover.RandomChoiceMover at 0x125fc4ba8>}